# running BenediktSeidl's code
This doesn't run in notebooks. check the [the script version](playwright-test.py). The error is the same in the next code blocks

In [ ]:
from playwright.sync_api import sync_playwright
with sync_playwright() as p:
    p.firefox.launch()

Error: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.

# example of path not working

In [15]:
from playwright.async_api import async_playwright
async with async_playwright() as p:
    browser = await p.firefox.launch(headless=False)
    page    = await browser.new_page()
    await page.goto("https://nixos.org")
    print("Title:", await page.title())
    await browser.close()

Error: BrowserType.launch: Executable doesn't exist at /nix/store/42axmwmwydsh8kzznd1mz7lnmg0a1zkk-playwright-browsers/firefox-1488/firefox/firefox
╔════════════════════════════════════════════════════════════╗
║ Looks like Playwright was just installed or updated.       ║
║ Please run the following command to download new browsers: ║
║                                                            ║
║     playwright install                                     ║
║                                                            ║
║ <3 Playwright Team                                         ║
╚════════════════════════════════════════════════════════════╝

In [16]:
from playwright.async_api import async_playwright
async with async_playwright() as p:
    browser = await p.chromium.launch(headless=False)
    page    = await browser.new_page()
    await page.goto("https://nixos.org")
    print("Title:", await page.title())
    await browser.close()

Error: BrowserType.launch: Executable doesn't exist at /nix/store/42axmwmwydsh8kzznd1mz7lnmg0a1zkk-playwright-browsers/chromium-1179/chrome-linux/chrome
╔════════════════════════════════════════════════════════════╗
║ Looks like Playwright was just installed or updated.       ║
║ Please run the following command to download new browsers: ║
║                                                            ║
║     playwright install                                     ║
║                                                            ║
║ <3 Playwright Team                                         ║
╚════════════════════════════════════════════════════════════╝

# setting custom executable paths
If I set a custom executable path, playwright code will work.

The python playwright api will store the available browsers in `./.venv/lib/python3.13/site-packages/playwright/driver/package/browsers.json`.

```json
{
  "comment": "Do not edit this file, use utils/roll_browser.js",
  "browsers": [
    {
      "name": "chromium",
      "revision": "1179",
      "installByDefault": true,
      "browserVersion": "138.0.7204.23"
    },
    {
      "name": "chromium-headless-shell",
      "revision": "1179",
      "installByDefault": true,
      "browserVersion": "138.0.7204.23"
    },
    {
      "name": "chromium-tip-of-tree",
      "revision": "1337",
      "installByDefault": false,
      "browserVersion": "139.0.7216.0"
    },
    {
      "name": "chromium-tip-of-tree-headless-shell",
      "revision": "1337",
      "installByDefault": false,
      "browserVersion": "139.0.7216.0"
    },
    {
      "name": "firefox",
      "revision": "1488",
      "installByDefault": true,
      "browserVersion": "139.0"
    },
    {
      "name": "firefox-beta",
      "revision": "1482",
      "installByDefault": false,
      "browserVersion": "138.0b10"
    },
    {
      "name": "webkit",
      "revision": "2182",
      "installByDefault": true,
      "revisionOverrides": {
        "debian11-x64": "2105",
        "debian11-arm64": "2105",
        "mac10.14": "1446",
        "mac10.15": "1616",
        "mac11": "1816",
        "mac11-arm64": "1816",
        "mac12": "2009",
        "mac12-arm64": "2009",
        "mac13": "2140",
        "mac13-arm64": "2140",
        "ubuntu20.04-x64": "2092",
        "ubuntu20.04-arm64": "2092"
      },
      "browserVersion": "18.5"
    },
    {
      "name": "ffmpeg",
      "revision": "1011",
      "installByDefault": true,
      "revisionOverrides": {
        "mac12": "1010",
        "mac12-arm64": "1010"
      }
    },
    {
      "name": "winldd",
      "revision": "1007",
      "installByDefault": false
    },
    {
      "name": "android",
      "revision": "1001",
      "installByDefault": false
    }
  ]
}
```

If we follow the instructions to set a environment variable of `PLAYWRIGHT_BROWSERS_PATH =  "${pkgs.playwright-driver.browsers}";`. The latest brower versions stored in the directory will be out of date.

if I run a ls for the variable `PLAYWRIGHT_BROWSERS_PATH `, it will show out of date browsers, which will cause the errors above

```bash
ls $PLAYWRIGHT_BROWSERS_PATH 
chromium-1169@  chromium_headless_shell-1169@  ffmpeg-1011@  firefox-1482@  webkit-2158@
```

In the next couple of cells, I will show a workaround, but it doesn't work if playwright is embedded as a library.

In [8]:
import os
playwright_browsers_path = os.environ['PLAYWRIGHT_BROWSERS_PATH']
playwright_browsers_path

'/nix/store/42axmwmwydsh8kzznd1mz7lnmg0a1zkk-playwright-browsers'

In [10]:
firefox_executable_path = os.path.join(playwright_browsers_path, "firefox-1482", "firefox", "firefox")
firefox_executable_path

'/nix/store/42axmwmwydsh8kzznd1mz7lnmg0a1zkk-playwright-browsers/firefox-1482/firefox/firefox'

In [11]:
from playwright.async_api import async_playwright
async with async_playwright() as p:
    browser = await p.firefox.launch(headless=False, executable_path=firefox_executable_path)
    page    = await browser.new_page()
    await page.goto("https://nixos.org")
    print("Title:", await page.title())
    await browser.close()

Title: Nix & NixOS | Declarative builds and deployments


In [ ]:
chromium_executable_path = os.path.join(playwright_browsers_path, "chromium-1169", "chrome-linux", "chrome")
chromium_executable_path

In [14]:
from playwright.async_api import async_playwright
async with async_playwright() as p:
    browser = await p.chromium.launch(headless=False, executable_path=chromium_executable_path)
    page    = await browser.new_page()
    await page.goto("https://nixos.org")
    print("Title:", await page.title())
    await browser.close()

Title: Nix & NixOS | Declarative builds and deployments
